<a href="https://colab.research.google.com/github/za2111/Bot-pcyholog/blob/main/bot_pcyh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Документация бота-психолога

## **1. Описание**  
Бот предоставляет психологическую поддержку, используя **GPT-4o-mini** через API `g4f`. Он имитирует работу профессионального психолога:  
- Поддерживает диалог с контекстом (история сообщений).  
- Отвечает мягко и эмпатично.  
- Имеет функции управления историей переписки.  

---

## **2. Установка и запуск**  

### **Требования**  
- Python 3.9+  
- Установленные зависимости(2 строка кода в google colab):
``aiogram g4f nest_asyncio``
- env файл с токеном бота(3 строка кода в google colab).

### **Запуск**

4 строка в google colab

**Либо**

``python bot.pcyh.py``

---

## **3. Функционал**  

### **Команды**  
| Команда | Описание |  
|---------|----------|  
| `/start` | Начать диалог (очищает историю) |  

### **Инлайн-кнопки**  
- **"Очистить историю"** – удаляет контекст диалога.  
- **"Последние 5 сообщений"** – показывает сокращенную историю.  

### **Автонапоминания**  
Если пользователь не пишет **10 минут**, бот отправляет сообщение:  
> *"Привет! 😊 Давно не общались, хотите продолжить разговор?"*  

---

## **4. Примеры диалогов**  

### **Пример 1: Тревога**  
```
👤 *Я не могу уснуть из-за тревоги...*  
🧠 **Понимаю, что тревога может сильно мешать заснуть. Может, поможет несколько расслабляющих техник? Например, глубокое дыхание или медитация. Попробуй вдохнуть на счёт 4, задержать дыхание на 4 секунды и выдохнуть на 6. Повторяй несколько раз, пока не почувствуешь облегчение. Ты уже пробовал какие-то способы расслабления перед сном?**  
```  

### **Пример 2: Конфликт**  
```
👤 *Мы поссорились с другом, и теперь мне стыдно.*  
🧠 *Это действительно тяжело, когда возникают конфликты с близкими людьми. Стыд — очень человеческое чувство, и важно его осознавать. Иногда мы можем сказать или сделать что-то, что не отражает нашу истинную природу. Постарайся понять, что именно вызвало у тебя это чувство. Может быть, есть возможность обсудить ситуацию с другом и поделиться своими переживаниями? Признание своих чувств и открытость могут помочь восстановить связь. Как ты сам думаешь, что могло бы помочь вам в этой ситуации?*  
```  

---

## **5. Логирование**  
В боте присутствует логирование которое сохраняет все в файл bot.log:  
```log
2025-08-16 14:00:00 - INFO - Бот запущен ✅  
2025-08-16 14:05:00 - WARNING - Timeout для пользователя 12345  
```  

---

## **6. Ограничения**  
- Максимальная длина истории: 20 сообщений.  
- Таймаут запроса к GPT: 30 секунд.  
- Для работы g4f требуется стабильный интернет.  

---

##          **Дополнительно**  
- Можно изменить SYSTEM_PROMPT для настройки стиля ответов.

- Поставьте 90+ пожалуйста 😫🙏🙏💓

In [1]:
# Установите зависимости

!pip install aiogram g4f nest_asyncio
print("Зависимости установлены!🫠")

Зависимости установлены!🫠


In [13]:
# Установите токен бота вместо "ВАШ_ТОКЕН_БОТА"


import os

os.environ["TELEGRAM_TOKEN"] = "ВАШ_ТОКЕН_БОТА"
API_TOKEN = os.environ.get("TELEGRAM_TOKEN")
if not API_TOKEN or API_TOKEN == "ВАШ_ТОКЕН_БОТА":
    raise RuntimeError("Установите TELEGRAM_TOKEN через переменные окружения")
else:
    print("Токен установлен!👀")

RuntimeError: Установите TELEGRAM_TOKEN через переменные окружения

In [3]:
# Запустите бота

# ---------------- Импорты ----------------

import os
import nest_asyncio
import asyncio
import logging
from datetime import datetime, timedelta

from aiogram import Bot, Dispatcher, types
from aiogram.types import Message, InlineKeyboardMarkup, InlineKeyboardButton
from aiogram.filters import Command
import g4f

# ---------------- Настройка логирования ----------------
logging.basicConfig(
    level=logging.INFO,
    filename="bot.log",
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# ---------------- Настройка бота ----------------
API_TOKEN = os.environ.get("TELEGRAM_TOKEN")
if not API_TOKEN:
    raise RuntimeError("Установите TELEGRAM_TOKEN через переменные окружения")

nest_asyncio.apply()
bot = Bot(token=API_TOKEN)
dp = Dispatcher()

# ---------------- Настройки истории ----------------
MAX_HISTORY = 20
user_histories = {}
user_last_active = {}

SYSTEM_PROMPT = (
    "Вы профессиональный психолог. "
    "Отвечайте исключительно на русском языке. "
    "Проявляйте эмпатию, внимательность и мягкость. "
    "Старайтесь помогать пользователю осознать свои эмоции и найти решения, "
    "не давая резких советов и поддерживая доверительную атмосферу."
)

# ---------------- Работа с историей ----------------
def append_history(user_id, role, content):
    if user_id not in user_histories:
        user_histories[user_id] = [{"role": "system", "content": SYSTEM_PROMPT}]
    user_histories[user_id].append({"role": role, "content": content})
    if len(user_histories[user_id]) > MAX_HISTORY + 1:
        user_histories[user_id] = [user_histories[user_id][0]] + user_histories[user_id][-MAX_HISTORY:]

# ---------------- Генерация ответа ----------------
async def get_ai_response(user_id: int, user_text: str) -> str:
    append_history(user_id, "user", user_text)
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + user_histories[user_id][1:]

    try:
        response = await asyncio.wait_for(
            asyncio.to_thread(g4f.ChatCompletion.create, model="gpt-4o-mini", messages=messages),
            timeout=30
        )
        append_history(user_id, "assistant", response)
        return response
    except asyncio.TimeoutError:
        logging.warning(f"Timeout для пользователя {user_id}")
        return "Сервис недоступен, попробуйте позже ⏳"
    except Exception as e:
        logging.error(f"Ошибка генерации ответа: {e}")
        return f"Произошла ошибка при генерации ответа: {e}"

# ---------------- Клавиатура ----------------
def get_main_keyboard():
    return InlineKeyboardMarkup(inline_keyboard=[
        [
            InlineKeyboardButton(text="Очистить историю", callback_data="reset_history"),
            InlineKeyboardButton(text="Последние 5 сообщений", callback_data="last5")
        ]
    ])

# ---------------- Старт ----------------
@dp.message(Command("start"))
async def start(message: Message):
    user_id = message.from_user.id
    user_histories[user_id] = [{"role": "system", "content": SYSTEM_PROMPT}]
    user_last_active[user_id] = datetime.now()

    await message.answer(
        "Привет! Я ваш психолог 🧠✨\n"
        "Расскажите о вашем случае, и я постараюсь помочь.\n\n",
        reply_markup=get_main_keyboard()
    )

# ---------------- Обработка кнопок ----------------
@dp.callback_query()
async def handle_callback(query: types.CallbackQuery):
    user_id = query.from_user.id

    if query.data == "reset_history":
        user_histories[user_id] = [{"role": "system", "content": SYSTEM_PROMPT}]
        await query.message.edit_text("История переписки очищена. Начнем новый диалог 🆕", reply_markup=get_main_keyboard())

    elif query.data == "last5":
        history_text = ""
        for entry in user_histories[user_id][-5:]:
            role = "👤 Вы" if entry["role"] == "user" else "🧠 Психолог"
            history_text += f"{role}: {entry['content']}\n\n"
        await query.message.answer(history_text, reply_markup=get_main_keyboard())

# ---------------- Обработка сообщений ----------------
@dp.message()
async def handle_message(message: Message):
    user_id = message.from_user.id
    user_last_active[user_id] = datetime.now()

    thinking_msg = await message.answer("💭 хм-хм, обдумываю ваш случай...")
    ai_response = await get_ai_response(user_id, message.text)
    await thinking_msg.edit_text(f"🧠 Психолог: {ai_response}", reply_markup=get_main_keyboard())

# ---------------- Таймер напоминаний ----------------
async def reminder_loop():
    while True:
        now = datetime.now()
        for user_id, last_time in user_last_active.items():
            if now - last_time > timedelta(minutes=10):
                try:
                    await bot.send_message(
                        user_id,
                        "Привет! 😊 Давно не общались, хотите продолжить разговор?",
                        reply_markup=get_main_keyboard()
                    )
                    user_last_active[user_id] = now
                except Exception as e:
                    logging.warning(f"Не удалось отправить напоминание пользователю {user_id}: {e}")
        await asyncio.sleep(60)

# ---------------- Запуск ----------------
if __name__ == "__main__":
    print("Бот запущен ✅")
    loop = asyncio.get_event_loop()
    loop.create_task(reminder_loop())
    loop.run_until_complete(dp.start_polling(bot))

Бот запущен ✅
g4f is up-to-date (version 0.6.0.1).
